## Imports

In [1]:
import PyPDF2 #Reads PDF Files
import ujson
import hashlib
import pickle
from unidecode import unidecode

# Initializing Master Repository of Content
### Starting from the complete PDF Library being downloaded and the binary file of each PDF document hashed and appended to a JSON object for that file

In [2]:
#Recreate complete list of JSON objects that were created:
working_uid_list = []
working_hash_dict = {}

for x in range (100, 37871):
    try:
        with open ('D:\BSA_PDF_Files\JSON\BSA_{}.json'.format(x),'r') as j:
            temp_hash = ujson.load(j)["Hash"]
            working_uid_list.append('BSA_{}'.format(x))
            if temp_hash in working_hash_dict:
                working_hash_dict[temp_hash].append('BSA_{}'.format(x))
            else:
                working_hash_dict.update({temp_hash:['BSA_{}'.format(x)]})
    except:
        continue
print(len(working_uid_list))
print(len(working_hash_dict))

36549
5546


In [3]:
def convertPDF2String(path):
    content = ""
    # load pdf file
    pdf = PyPDF2.PdfFileReader(file(path,"rb"))
    if pdf.isEncrypted: 
        try: pdf.decrypt('')
        except: print("Attempted decrpytion failed on UID: "+uid)
    # iterate pages
    for i in range(0, pdf.getNumPages()):
        # extract the text from each page
        try: content += pdf.getPage(i).extractText() + " \n"
        except: continue
    # collapse whitespaces
    content = " ".join(content.replace(u"\xa0", " ").split()).encode('utf-8')
    return (content) #.encode('utf-8')

#Append textual content to dictionary for ecah unique PDF file
def contentReadAndAppend(uid):
    try: 
        with open('D:\BSA_PDF_Files\JSON\\'+uid+'.json', 'r') as i:
            uid_dict_check = ujson.load(i)
            old_content = uid_dict_check["Content"]
    except: 
        old_content = None
    if old_content is None:
        content = convertPDF2String('D:\BSA_PDF_Files\\'+uid+'.pdf')
        with open('D:\BSA_PDF_Files\JSON\\'+uid+'.json', 'r') as j: #open JSON for the file in question
            uid_dict = ujson.load(j)
        uid_dict.update({"Content":content})
        with open('D:\BSA_PDF_Files\JSON\\'+uid+'.json', 'w') as k:
            ujson.dump(uid_dict,k,indent=4)

def addPageNumbers(uid):
    try:
        with open ('D:\BSA_PDF_Files\JSON\\'+uid+'.json','r') as j:
            uid_dict_check = ujson.load(i)
            old_pgn = uid_dict_check["Pages"]
    except:
        old_pgn = None
    if old_pgn is None:
        tempPdf = PyPDF2.PdfFileReader(file('D:\BSA_PDF_Files\\'+uid+'.pdf','rb'))
        if tempPdf.isEncrypted: 
                    try: tempPdf.decrypt('')
                    except: print("Attempted decrpytion failed on uid: "+uid)
        pageNumbers = tempPdf.getNumPages()
        #print (pageNumbers)
        with open('D:\BSA_PDF_Files\JSON\\'+uid+'.json', 'r') as k:
            uid_dict = ujson.load(k)
        uid_dict.update({"Pages":pageNumbers})
        with open('D:\BSA_PDF_Files\JSON\\'+uid+'.json', 'w') as l:
            ujson.dump(uid_dict,l,indent=4)

for uid in working_uid_list:
    try:contentReadAndAppend(uid)
    except:print("Content Error on UID: "+uid)
    try:addPageNumbers(uid)
    except:print("PageNum Error on UID: "+uid)

In [10]:
content_hash_dict = {}

def create_content_hash(uid):
    global content_hash_dict
    try:
        with open ('D:\BSA_PDF_Files\JSON\\'+uid+'.json','r') as j:    
            uid_dict = ujson.load(j)
        uid_content = uid_dict['Content']
        
    except:
        uid_content = None    
    #print(uid_content)
    
    if uid_content is not None: 
        try:
            tempHash = hashlib.sha1(uid_content).hexdigest()
        except:
            decoded_content = unidecode(uid_content)
            tempHash = hashlib.sha1(decoded_content).hexdigest()
        #print(tempHash)
        if tempHash in content_hash_dict:
            content_hash_dict[tempHash].append(uid) #add new has to global dictionary
        else:
            content_hash_dict.update({tempHash:[uid]}) #append UID to an existing list of UIDs for that hash value
        
        uid_dict.update({"Content_Hash":tempHash})

    with open('D:\BSA_PDF_Files\JSON\\'+uid+'.json','w') as k:
        ujson.dump(uid_dict,k,indent=4)


for uid in working_uid_list:
    create_content_hash(uid)
        
print(len(content_hash_dict))

with open('D:\BSA_PDF_Files\JSON\Content_Hash_Dict.json','w') as o:
    ujson.dump(content_hash_dict,o,indent=4)

4222


In [59]:
with open ("working_uid_list.pickle","w") as u:
    pickle.dump(working_uid_list, u)
with open ("working_hash_dict.pickle","w") as v:
    pickle.dump(working_hash_dict, v)
with open ("content_hash_dict.pickle","w") as x:
    pickle.dump(content_hash_dict, x)
with open('bin_hash_url.pickle','w') as z:
    pickle.dump(bin_hash_url,z)

In [2]:
working_uid_list =  pickle.load(open('working_uid_list.pickle','r'))
working_hash_dict = pickle.load(open('working_hash_dict.pickle','r'))
content_hash_dict = pickle.load(open('content_hash_dict.pickle','r'))
bin_hash_url = pickle.load(open('bin_hash_url.pickle','r'))

In [3]:
print(len(working_uid_list))
print(len(working_hash_dict))
print(len(content_hash_dict))
print(len(bin_hash_url))

36549
5546
4221
5546


In [50]:
bin_hash_url = {}
for bin_hash in working_hash_dict:
    
    uids = working_hash_dict[bin_hash]
    urls = []
    for uid in uids:
        with open ('D:\BSA_PDF_Files\JSON\\'+uid+'.json','r') as j:    
            urls.append(ujson.load(j)["URL"])
    bin_hash_url.update({bin_hash:urls})
with open ("bin_hash_url.pickle","w") as d:
    pickle.dump(bin_hash_url, d)

print("Operation Complete")


Operation Complete


In [52]:
with open('Hash_to_URLs.json','w') as f:
    ujson.dump(bin_hash_url,f, indent = 4)

## Attempt to initialize and use MongoDB for the files

In [3]:
from pymongo import MongoClient
# Connect to the mongo local database
connection = MongoClient()
db = connection.bsa_files
collection = db.bsa_files

In [3]:
print(connection)
print
print(db)
print
print(connection.database_names)
print
print(collection)
print
print(db.collection_names)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'bsa_files')

<bound method MongoClient.database_names of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'bsa_files'), u'bsa_files')

<bound method Database.collection_names of Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'bsa_files')>


In [7]:
with open('D:\BSA_PDF_Files\JSON\BSA_101.json','r') as j:
    test_file = ujson.load(j)
print(test_file)

{u'Domain': u'adventure.oa-bsa.org', u'Hash': u'bd1555e13f710c5ebf508349bc97fd87cec1e69f', u'UID': u'BSA_101', u'URL': u'http://adventure.oa-bsa.org/files/flyers/GENERAL_OAHA_FLYER.pdf', u'File_Name': u'GENERAL_OAHA_FLYER.pdf', u'Content': u'ORDER OF THE ARROW HIGH ADVENTURE ADVENTURE.OA - BSA.ORG TRAIL CREW WILDERNESS VOYAGE CANADIAN ODYSSEY OCEAN ADVENTURE SUMMIT EXPERIENCE Requirements: Be at least 16 years old by the time your program starts but not yet 21 by its conclusion(14 - 17 for Summit Experience) Be active in the BSA and a local OA Lodge See applications for height and weight restrictions', u'Content_Hash': u'83602de48c5d4ce405560dbaebeff9a3091acbe3', u'File_Size': 400338, u'Folder': u'flyers', u'Pages': 1, u'Metadata': {u'/Creator': u'Microsoft\xae Publisher 2013', u'/ModDate': u"D:20150503103301-05'00'", u'/Producer': u'Microsoft\xae Publisher 2013', u'/CreationDate': u"D:20150503103301-05'00'", u'/Author': u'sean ferrier'}}


In [8]:
collection.insert_one(test_file)

In [38]:
retrieved_file = collection.find_one({"UID":"BSA_101"})
print(retrieved_file)

{u'Domain': u'adventure.oa-bsa.org', u'Hash': u'bd1555e13f710c5ebf508349bc97fd87cec1e69f', u'UID': u'BSA_101', u'Language': u'english', u'URL': u'http://adventure.oa-bsa.org/files/flyers/GENERAL_OAHA_FLYER.pdf', u'File_Name': u'GENERAL_OAHA_FLYER.pdf', u'Pages': 1, u'Content': u'ORDER OF THE ARROW HIGH ADVENTURE ADVENTURE.OA - BSA.ORG TRAIL CREW WILDERNESS VOYAGE CANADIAN ODYSSEY OCEAN ADVENTURE SUMMIT EXPERIENCE Requirements: Be at least 16 years old by the time your program starts but not yet 21 by its conclusion(14 - 17 for Summit Experience) Be active in the BSA and a local OA Lodge See applications for height and weight restrictions', u'File_Size': 400338, u'Content_Hash': u'83602de48c5d4ce405560dbaebeff9a3091acbe3', u'_id': ObjectId('582cbfb88726e30d949a9815'), u'Folder': u'flyers', u'Metadata': {u'/CreationDate': u"D:20150503103301-05'00'", u'/Author': u'sean ferrier', u'/Producer': u'Microsoft\xae Publisher 2013', u'/Creator': u'Microsoft\xae Publisher 2013', u'/ModDate': u"D:2

In [44]:
print(collection.find_one({'Language':'english'}))
print('*'*75)
print(collection.find_one({'Language':'spanish'}))

{u'Domain': u'adventure.oa-bsa.org', u'Hash': u'bd1555e13f710c5ebf508349bc97fd87cec1e69f', u'UID': u'BSA_101', u'Language': u'english', u'URL': u'http://adventure.oa-bsa.org/files/flyers/GENERAL_OAHA_FLYER.pdf', u'File_Name': u'GENERAL_OAHA_FLYER.pdf', u'Pages': 1, u'Content': u'ORDER OF THE ARROW HIGH ADVENTURE ADVENTURE.OA - BSA.ORG TRAIL CREW WILDERNESS VOYAGE CANADIAN ODYSSEY OCEAN ADVENTURE SUMMIT EXPERIENCE Requirements: Be at least 16 years old by the time your program starts but not yet 21 by its conclusion(14 - 17 for Summit Experience) Be active in the BSA and a local OA Lodge See applications for height and weight restrictions', u'File_Size': 400338, u'Content_Hash': u'83602de48c5d4ce405560dbaebeff9a3091acbe3', u'_id': ObjectId('582cbfb88726e30d949a9815'), u'Folder': u'flyers', u'Metadata': {u'/CreationDate': u"D:20150503103301-05'00'", u'/Author': u'sean ferrier', u'/Producer': u'Microsoft\xae Publisher 2013', u'/Creator': u'Microsoft\xae Publisher 2013', u'/ModDate': u"D:2

In [23]:
def write_JSON_to_mongo(uid):
    with open('D:\BSA_PDF_Files\JSON\{}.json'.format(uid),'r') as j:
        collection.insert_one(ujson.load(j))

for uid in working_uid_list:
    write_JSON_to_mongo(uid)

In [6]:
import nltk

english_stopwords = nltk.corpus.stopwords.words('english')
spanish_stopwords = nltk.corpus.stopwords.words('spanish')

common_stopwords = list(set(english_stopwords) & set(spanish_stopwords))
print common_stopwords

[u'me', u'a', u'no', u'o', u'y', u'has', u'he']


In [43]:
def set_language(uid):
    language = None
    record = collection.find_one({'UID':uid})
    if record["Content"]!= None and record["Content"]!= '':
        if "Language" not in record:
            if len(list(set(english_stopwords) & set(record["Content"].strip().split()))) >= len(list(set(spanish_stopwords) & set(record["Content"].strip().split()))):
                language = "english"
            elif len(list(set(english_stopwords) & set(record["Content"].strip().split()))) < len(list(set(spanish_stopwords) & set(record["Content"].strip().split()))):
                language = "spanish"
            else:
                language = "unknown"
            result = collection.update_one({"UID":uid},{"$set":{"Language":language}})                  
            #print(result.modified_count)
        elif record["Language"]== None:
            if len(list(set(english_stopwords) & set(record["Content"].strip().split()))) >= len(list(set(spanish_stopwords) & set(record["Content"].strip().split()))):
                language = "english"
            elif len(list(set(english_stopwords) & set(record["Content"].strip().split()))) < len(list(set(spanish_stopwords) & set(record["Content"].strip().split()))):
                language = "spanish"
            else:
                language = "unknown"
            result = collection.update_one({"UID":uid},{"$set":{"Language":language}})                  
            #print(result.modified_count)
    
for uid in working_uid_list:
    try:
        set_language(uid)
    except:
        print uid
    
print "language update completed"

language update completed
